In [ ]:
!pip install optuna -q
!pip install --upgrade datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive') # mounting drive where files are stored

Mounted at /content/drive


In [ ]:
import sys
path = "/content/drive/MyDrive/AI_Cybersecurity/Project/"
sys.path.append(path)

In [ ]:
import pandas as pd
from utils import urls_to_string, BPE_Tokenizer, find_bin, add_stratification_column, train_val_test_split, add_metadata
import json
import regex as re
import numpy as np
from datasets import Dataset, ClassLabel
from collections import defaultdict
import base64
import ast

In [ ]:
urls = pd.read_csv(path+"PhiUSIIL_Phishing_URL_Dataset.csv")
urls

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.000000,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.000000,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.000000,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.000000,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.000000,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235790,660997.txt,https://www.skincareliving.com,29,www.skincareliving.com,22,0,com,100.000000,1.000000,0.522907,...,1,0,1,51,7,21,187,2,191,1
235791,77185.txt,https://www.winchester.gov.uk,28,www.winchester.gov.uk,21,0,uk,100.000000,0.785714,0.028555,...,1,0,0,50,1,7,88,0,31,1
235792,622132.txt,https://www.nononsensedesign.be,30,www.nononsensedesign.be,23,0,be,100.000000,1.000000,0.003319,...,0,0,1,27,10,30,58,2,67,1
235793,7503962.txt,https://patient-cell-40f5.updatedlogmylogin.wo...,55,patient-cell-40f5.updatedlogmylogin.workers.dev,47,0,dev,28.157537,0.465116,0.000961,...,0,0,0,0,0,3,0,0,0,0


In [ ]:
text_feature_columns = urls.select_dtypes(include=['object', 'string']).columns.tolist()  # Text-based columns (URLs)
numerical_columns = urls.select_dtypes(include=['int64', 'float64']).columns.tolist()  # Numerical columns
target_column = 'label'  # The target column for classification
if target_column in text_feature_columns:
    text_feature_columns.remove(target_column)
if target_column in numerical_columns:
    numerical_columns.remove(target_column)

print("The text feature columns are:", text_feature_columns)
print("The numerical columns are:", numerical_columns)
print("The target column is:", target_column)

The text feature columns are: ['FILENAME', 'URL', 'Domain', 'TLD', 'Title']
The numerical columns are: ['URLLength', 'DomainLength', 'IsDomainIP', 'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']
The target column is: label


In [ ]:
# Create the string
url_string = urls_to_string(urls['URL'])
title_string = urls_to_string(urls['Title'])

# Output the result
print(url_string[:1000])
print(title_string[:1000])

https://www.southbankmosaics.com https://www.uni-mainz.de https://www.voicefmradio.co.uk https://www.sfnmjournal.com https://www.rewildingargentina.org https://www.globalreporting.org https://www.saffronart.com https://www.nerdscandy.com https://www.hyderabadonline.in https://www.aap.org https://www.religionenlibertad.com http://www.teramill.com https://www.socialpolicy.org https://www.aoh61.com https://www.bulgariaski.com https://www.brightika.com https://www.motley.ie https://www.funzine.hu https://www.dixxon.com https://www.ooty.ind.in http://www.f0519141.xsph.ru http://www.shprakserf.gq https://www.cryptocompare.com https://www.diariodealmeria.es https://www.town.minamichita.lg.jp https://www.bwresearch.com https://www.musicvideoproduction.guru https://service-mitld.firebaseapp.com/ http://www.kuradox92.lima-city.de https://liuy-9a930.web.app/ https://www.landed.com https://ipfs.io/ipfs/qmrvvyr84esa2assw9vvwupqjgsdn4c3dwkusfdwzdz3kn?clientid=noc@protocol.ai http://att-103731-107123

In [ ]:
final_text = url_string + " " + title_string

In [ ]:
z = BPE_Tokenizer(text=final_text, vocab_size=260, notify_every=1)
z.save_vocab(path+'Test_Tokenizer')
print(z.vocab)

print(z.decode(z.encode(urls['URL'][0])) == urls['URL'][0])

Notification: Vocab ID 256 has been created.
Notification: Vocab ID 257 has been created.
Notification: Vocab ID 258 has been created.
Notification: Vocab ID 259 has been created.
{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I',

In [ ]:
examples = [f"<URL>https://example.com <TITLE>Example Title", f"<URL>{urls['URL'][0]} <TITLE>{urls['Title'][0]}", f"<URL>{urls['URL'][1]} <TITLE>{urls['Title'][1]}"]
decoded_examples = [f"https://example.com Example Title", f"{urls['URL'][0]} {urls['Title'][0]}", f"{urls['URL'][1]} {urls['Title'][1]}"]

print("Decoded:", decoded_examples)

print(z.decode(z.encode(examples)) == decoded_examples)

Decoded: ['https://example.com Example Title', 'https://www.southbankmosaics.com à¸‚à¹ˆà¸²à¸§à¸ªà¸” à¸‚à¹ˆà¸²à¸§à¸§à¸±à¸™à¸™à¸µà¹‰ à¸‚à¹ˆà¸²à¸§à¸\x81à¸µà¸¬à¸² à¸‚à¹ˆà¸²à¸§à¸šà¸±à¸™à¹€à¸—à¸´à¸‡ à¸\xadà¸±à¸žà¹€à¸”à¸—à¸ªà¸”à¹ƒà¸«à¸¡à¹ˆà¸—à¸¸à¸\x81à¸§à¸±à¸™ &#8211; à¸‚à¹ˆà¸²à¸§à¸ªà¸” à¸‚à¹ˆà¸²à¸§à¸\x81à¸µà¸¬à¸² à¸‚à¹ˆà¸²à¸§à¸šà¸±à¸™à¹€à¸—à¸´à¸‡ à¸‚à¹ˆà¸²à¸§à¸§à¸±à¸™à¸™à¸µà¹‰ à¸\xadà¸±à¸›à¹€à¸”à¸•à¸‚à¹ˆà¸²à¸§à¸ªà¸²à¸£à¸£à¸§à¸”à¹€à¸£à¹‡à¸§à¸—à¸±à¸™à¹ƒà¸ˆ à¸žà¸£à¹‰à¸\xadà¸¡à¸£à¸±à¸šà¸Šà¸¡à¸ªà¸²à¸£à¸°à¸™à¹ˆà¸²à¸£à¸¹à¹‰à¸•à¹ˆà¸²à¸‡à¹† à¹„à¸”à¹‰à¸Ÿà¸£à¸µà¸•à¸¥à¸\xadà¸” 24à¸Šà¸±à¹ˆà¸§à¹‚à¸¡à¸‡', 'https://www.uni-mainz.de johannes gutenberg-universitÃ¤t mainz']
True


In [ ]:
# try:
#     tokenizer = BPE_Tokenizer(text=final_text, vocab_size=10000, notify_every=100, save_every=100, save_directory='C:/Users/Lenovo/Documents/Project/Final_Tokenizer')
# except Exception as e:
#     print(f"An error occurred during vocab creation: {e}")
# finally:
#     # Ensure the vocab is saved even if an error occurs
#     tokenizer.save_vocab('C:/Users/Lenovo/Documents/Project/Final_Tokenizer')

In [ ]:
tokenizer = BPE_Tokenizer(directory=path+'Tokenizer')
for key, value in tokenizer.vocab.items():
    print(f"{key}: {value}")

Streaming output truncated to the last 5000 lines.
5006: b'clay'
5007: b'677'
5008: b'mim'
5009: b'999'
5010: b'usiness'
5011: b'856'
5012: b'omg'
5013: b'tea'
5014: b'ethe'
5015: b'343'
5016: b'ogy'
5017: b'amazzeocn'
5018: b'epo'
5019: b'800'
5020: b'uac'
5021: b'env'
5022: b'eweb'
5023: b'apantheonsite'
5024: b'aac'
5025: b'smtb'
5026: b'itsa'
5027: b'plug'
5028: b'464'
5029: b'tus'
5030: b'alab'
5031: b'explore'
5032: b'1040'
5033: b'preview'
5034: b'safety'
5035: b'allet'
5036: b'abar'
5037: b'littl'
5038: b'dae'
5039: b'answ'
5040: b'southern'
5041: b'ott'
5042: b'caseid'
5043: b'875'
5044: b'alle'
5045: b'girls'
5046: b'lion'
5047: b'crafts'
5048: b'ief'
5049: b'241'
5050: b'challenge'
5051: b'busines'
5052: b'yout'
5053: b'adli'
5054: b'comput'
5055: b'russ'
5056: b'010'
5057: b'ology'
5058: b'gift'
5059: b'floral'
5060: b'asz'
5061: b'african'
5062: b'beyond'
5063: b'madis'
5064: b'2020'
5065: b'aber'
5066: b'doub'
5067: b'stain'
5068: b'eff'
5069: b'282'
5070: b'pdh'
5071: b'

In [ ]:
url = urls['URL'][0]
tokenizer.decode(tokenizer.encode(url)) == url

True

In [ ]:
import pandas as pd
import re

def remove_special_tokens(text, special_tokens):
    """
    Removes specified special tokens from a given string, preserving trailing spaces.

    Args:
        text (str): The input string containing special tokens.
        special_tokens (list): A list of special tokens to be removed (e.g., "<URL>", "<TITLE>").

    Returns:
        str: The processed string with special tokens removed, preserving trailing spaces.
    """
    # Construct the regex pattern to match special tokens
    pattern = "|".join(re.escape(token) for token in special_tokens)

    # Remove special tokens, but do not strip trailing spaces
    return re.sub(pattern, "", text)

# List of special tokens
special_tokens = ["<URL>", "<TITLE>"]

In [ ]:
check = True
for url in urls['URL']:
    if tokenizer.decode(tokenizer.encode(url, return_attention_mask=False)) != url:
        print(f"{url} wasn't tokenized properly")
        check = False
if check == False:
    print("URLs tokenized properly")

check = True
for url in urls['Title']:
    if tokenizer.decode(tokenizer.encode(url, return_attention_mask=False)) != url:
        print(f"{url} wasn't tokenized properly")
        check = False
if check == False:
    print("Titles tokenized properly")

for index, row in urls.iterrows():
    # Format the example with special tokens
    example = f"<URL>{row['URL']} <TITLE>{row['Title']}"

    # Remove special tokens for the processed example
    processed_example = remove_special_tokens(example, special_tokens)

    # Encode and decode the example
    encoded = tokenizer.encode(example)
    decoded = tokenizer.decode(encoded)

    # Check if the decoded result matches the processed example
    is_equal = decoded == processed_example
    if not is_equal:
        print(f"{example} wasn't tokenized properly")
        check = False
# Define batch size
batch_size = 1024

# Create examples with special tokens
examples = [f"<URL>{row['URL']} <TITLE>{row['Title']}" for _, row in urls.iterrows()]

# Initialize correctness check
check = True
for i in range(0, len(examples), batch_size):
    # Get the current batch
    batch = examples[i:i + batch_size]

    # Tokenize the batch
    encoded_batch = tokenizer.encode(batch, add_special_tokens=True)

    # Decode the batch
    decoded_batch = tokenizer.decode(encoded_batch)

    # Remove special tokens from the original batch
    processed_batch = [remove_special_tokens(text, special_tokens) for text in batch]

    # Compare each processed example with the decoded example
    for processed, decoded in zip(processed_batch, decoded_batch):
        if processed.strip() != decoded.strip():
            print(f"Mismatch found in batch {i // batch_size}:\nProcessed: {processed}\nDecoded: {decoded}")
            check = False

if check == True:
    print("Input Texts tokenized properly")

In [ ]:
# Let's create a list to store the token lengths
token_lengths = []
total_length = 0

# Bins for token lengths
bins = [32, 64, 128, 256, 'beyond']  # Added 1024 as per previous discussion
bin_counts = {bin_size: 0 for bin_size in bins}

# Create a list to collect indices of URLs that don't exceed the maximum bin size (1024 tokens)
valid_indices = []

# Function to find the appropriate bin
def find_bin(length, bins):
    for bin_size in bins:
        if bin_size != 'beyond' and length <= bin_size:
            return bin_size
    return 'beyond'

# Iterate over each URL (or text example) in your dataset
for idx, row in urls.iterrows():
    example = f"<URL>{row['URL']} <TITLE>{row['Title']}"

    # Encode the URL using your tokenizer and extract 'input_ids'
    encoded_url = tokenizer.encode(example, return_attention_mask=False)
    tokenized_url = encoded_url['input_ids']  # Extract the 'input_ids'

    # Get the length of the tokenized URL
    length = len(tokenized_url)

    # Append the length of the tokenized URL to the list
    token_lengths.append((example, length))

    # Add the length to the total
    total_length += length

    # Find the appropriate bin for this length and increment its count
    bin_size = find_bin(length, bins)
    bin_counts[bin_size] += 1

    # Collect indices of URLs that don't exceed 1024 tokens (or any chosen threshold)
    if bin_size != 'beyond':
        valid_indices.append(idx)

# Find the URL with the maximum token length
longest_url, max_length = max(token_lengths, key=lambda x: x[1])

# Calculate the average token length
average_length = total_length / len(urls['URL'])

# Filter the dataset to include only URLs that are within the desired length
filtered_urls = urls.iloc[valid_indices]

# Output the longest tokenized example, its length, the average token length, and bin counts
print(f"Longest URL: {longest_url}")
print(f"Token Length: {max_length}")
print(f"Average Token Length: {average_length:.3f}")
print("Token Length Distribution (Bins):")
for bin_size, count in bin_counts.items():
    print(f"<= {bin_size} tokens: {count}")

print(f"Filtered dataset contains {len(filtered_urls)} URLs out of {len(urls)} total. {(len(urls) - len(filtered_urls)) / len(urls) * 100:.2f}% of the dataset removed")

Longest URL: <URL>https://bodhicardstech.com/beta/wx%20qz%20xx%20ww%20qxz%20wes%20wsed/54dr%2056yg%2065yft5%204f6yft5%20d46yg5%20fyt45t%20f4d4%20yf56td4/ft5%20fy%2065y%204%205tdr34%204yf46%20y5uyf%20t5%20d6yf5%20td%204fdtr34%205yf5%2064d5t34f46u75/gy6f5t%2056yf%205t%204g75%20fy65t%204%2065gf%20y%2054f%204g5ug7%20u5y6f4%205%20guf%20y6545d%204f6gu%207f65y%204dt/gy65ft%2045%20fy6%206g57guf%20y5%20f4%205%20y57%20yf6%204t5%20gu%207f5y645%20%20fu%20f5y%2064dt4tf/g4gr%20y65f%20f5yft4%2056yf%2045t%205gy%20f645ft%20475yf6%204tf5%20yft%2065ug7%2056fy45t%204%20u5fy6t%20f4u%207ug5y%204tf465%20u%206ug7y655/y65t%2056gy%2056gu6y%2045%20g7u%20y6f4%205g6u7u5%206yf4%2075gu6%20yf4%206gy5%20ft46u%207i6u7g5%20yf45%2056gu%20ug76%20y566f4%205u%2068%2067ug6y%20f4g4%2076ugy5%2045%20u76guy54/f54_4yg=u-y65f5t4t.46gu-g5=y64f-u5g7y6_46gu=-u6g75y6.3dt-46=u68gu-75_57hi6=8i-g6u7fyd4.u6-8i=6u56-yf45-46g=u57-86ug7f5y6d4.yf5-7u=86-u756y_46=g57h-8i6gu75.34rs-d5=y56-gu-6h=87i-h86gu756y.45yf-56u=68-gu75y-56gu6gu75y6/y65ft3

In [ ]:
print(f"There are {len(filtered_urls['label'])} total examples with {len(filtered_urls[filtered_urls['label'] == 0])} being classified as safe and {len(filtered_urls[filtered_urls['label'] == 0])} being classified as vulernable")

There are 233638 total examples with 98789 being classified as safe and 98789 being classified as vulernable


In [ ]:
urls_dataset = Dataset.from_pandas(filtered_urls)
urls_dataset.set_format(type='torch')

In [ ]:
# Apply the function to create a new 'stratify_label' column
urls_dataset = urls_dataset.map(add_stratification_column)

# Convert the 'stratify_label' column to ClassLabel type (required for stratification)
unique_labels = sorted(urls_dataset.unique('stratify_label'))  # Get unique class labels
class_labels = ClassLabel(names=[str(label) for label in unique_labels])  # Create ClassLabel object for stratification

# Cast the 'stratify_label' column to ClassLabel type
urls_dataset = urls_dataset.cast_column("stratify_label", class_labels)

In [ ]:
# Split the dataset into train, validation, and test sets with stratification
split_dataset = train_val_test_split(urls_dataset, train_val_ratio=0.2, val_test_ratio=0.5, stratify_column='stratify_label')

In [ ]:
def get_mean_stds(dataset, columns):
    stats = {}
    for col in columns:
        stats[col] = {
            'mean': dataset[col].float().mean(),
            'std': dataset[col].float().std()
        }
    return stats


# Step 2: Define a function to normalize the features, accepting mean and std as parameters
def normalize_function(examples, mean_std):
    for col in numerical_columns:
        examples[col] = (examples[col] - mean_std[col]['mean']) / (mean_std[col]['std'] + 1e-6)
    return examples

train_stats = get_mean_stds(split_dataset['train'], numerical_columns)

# Step 3: Apply the normalization to all splits using .map(), passing the train stats as parameters
normalized_dataset = split_dataset.map(
    normalize_function,
    fn_kwargs={'mean_std': train_stats},  # Pass the mean and std stats as arguments
    batched=True
)

Map: 100%|██████████| 23504/23504 [00:08<00:00, 2792.69 examples/s]


In [ ]:
prepared_dataset = normalized_dataset.map(lambda x: add_metadata(x, numerical_columns))

Map: 100%|██████████| 23504/23504 [01:12<00:00, 324.88 examples/s]


In [ ]:
cleaned_dataset = prepared_dataset.remove_columns(numerical_columns)
cleaned_dataset

DatasetDict({
    train: Dataset({
        features: ['FILENAME', 'URL', 'Domain', 'TLD', 'Title', 'label', '__index_level_0__', 'stratify_label', 'metadata'],
        num_rows: 188028
    })
    validation: Dataset({
        features: ['FILENAME', 'URL', 'Domain', 'TLD', 'Title', 'label', '__index_level_0__', 'stratify_label', 'metadata'],
        num_rows: 23504
    })
    test: Dataset({
        features: ['FILENAME', 'URL', 'Domain', 'TLD', 'Title', 'label', '__index_level_0__', 'stratify_label', 'metadata'],
        num_rows: 23504
    })
})

In [ ]:
cleaned_dataset.save_to_disk(path+"Prepared_Datasets/")

Saving the dataset (1/1 shards): 100%|██████████| 23504/23504 [00:00<00:00, 1056974.14 examples/s]


In [ ]:
# Tokenize and pad a batch of texts
texts = ["This is a sentence.", "This is another sentence, but it's longer."]
tokenized_and_padded_batch = tokenizer.enocde(texts)
print(tokenized_and_padded_batch)

NameError: name 'tokenizer' is not defined

: 